In [1]:
import pandas as pd
from scipy.io import loadmat
from collections import Counter

import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow as tf

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix

import warnings
warnings.filterwarnings(action='ignore')

df_spike = pd.DataFrame(loadmat("data/classification/spike_count_half.mat")['spike_count'])
df_target = pd.DataFrame(loadmat("data/classification/target_half.mat")['target']).T - 1
df_target.columns = ["y"]

df = pd.concat([df_spike, df_target], axis=1)
df

,0,1,2,3,4,5,6,7,8,9,...,89,90,91,92,93,94,95,96,97,y
0,2.0,2.0,6.0,6.0,0.0,0.0,13.0,0.0,4.0,0.0,...,4.0,3.0,2.0,0.0,1.0,0.0,10.0,0.0,5.0,0.0
1,2.0,2.0,4.0,9.0,0.0,1.0,9.0,0.0,3.0,0.0,...,4.0,2.0,0.0,2.0,2.0,0.0,13.0,2.0,6.0,0.0
2,4.0,2.0,3.0,7.0,0.0,0.0,8.0,0.0,3.0,0.0,...,5.0,1.0,1.0,0.0,2.0,0.0,12.0,0.0,1.0,0.0
3,3.0,1.0,8.0,9.0,1.0,0.0,15.0,0.0,2.0,0.0,...,5.0,5.0,2.0,1.0,3.0,0.0,11.0,3.0,4.0,0.0
4,3.0,3.0,7.0,11.0,0.0,1.0,10.0,0.0,4.0,0.0,...,1.0,3.0,1.0,0.0,1.0,2.0,8.0,2.0,2.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
795,1.0,0.0,0.0,9.0,2.0,2.0,8.0,0.0,2.0,0.0,...,5.0,6.0,1.0,1.0,1.0,1.0,10.0,0.0,1.0,7.0
796,0.0,1.0,1.0,4.0,0.0,2.0,11.0,0.0,4.0,0.0,...,4.0,11.0,0.0,0.0,1.0,0.0,4.0,1.0,1.0,7.0
797,1.0,0.0,4.0,11.0,1.0,3.0,15.0,0.0,5.0,0.0,...,7.0,6.0,1.0,1.0,1.0,2.0,10.0,0.0,1.0,7.0
798,1.0,0.0,3.0,7.0,0.0,3.0,15.0,0.0,1.0,0.0,...,7.0,6.0,2.0,1.0,0.0,1.0,14.0,0.0,4.0,7.0


In [2]:
X_train, X_test, y_train, y_test = train_test_split(df_spike, df_target, test_size=0.2, random_state=42, shuffle=True, stratify=df_target.y)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=42, shuffle=True)

print(f"X_train: {X_train.shape} X_valid: {X_valid.shape} X_test: {X_test.shape}")
print(f"y_test: {Counter(y_test.y.values)}")

X_train: (576, 98) X_valid: (64, 98) X_test: (160, 98)
y_test: Counter({5.0: 20, 2.0: 20, 6.0: 20, 3.0: 20, 0.0: 20, 1.0: 20, 4.0: 20, 7.0: 20})


In [3]:
def print_metrics(y_test, pred):
    accuracy = accuracy_score(y_test, pred)

    plt.figure(figsize=(7, 5))
    sns.heatmap(confusion_matrix(pred, y_test), annot=True, fmt="d")
    plt.show()
    
    print(f"Accuracy: {accuracy}")

In [ ]:
lm = LogisticRegression().fit(X_train, y_train)
pred = lm.predict(X_test)

print_metrics(y_test, pred)

In [ ]:
svc = SVC().fit(X_train, y_train)
pred = svc.predict(X_test)

print_metrics(y_test, pred)

In [ ]:
tree = DecisionTreeClassifier().fit(X_train, y_train)
pred = tree.predict(X_test)

print_metrics(y_test, pred)

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=[98]),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(8)
])

optimizer = tf.keras.optimizers.Adam(0.0001)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, verbose=0)
pred = tf.argmax(model.predict(X_test), axis=1).numpy()

print_metrics(y_test, pred)